In [67]:
import streamlit as st 
import pandas as pd
import numpy as npw3
import duckdb as db
from datetime import date, timedelta


In [68]:
file_id = "1F8ubYg-465HJtiMd-icUBqGVAGRUcAoK"
csv_url = f"https://docs.google.com/spreadsheets/d/{file_id}/export?format=csv"

df = pd.read_csv(csv_url)
columns = ['COURIER_INSENTIVE', 'Total Pickup']
existing_cols = [col for col in columns if col in df.columns]

df[existing_cols] = df[existing_cols].fillna(0).astype(int)
df.head()


,REF_AWB_NUMBER,STATUS_COD,PARTNER_NAME,SHIPPER_NAME,SHIPPER_ADDRESS,SHIPPER_CITY,SHIPPER_PROVINCE,DESTINATION_DISTRICT,DESTINATION_CITY,DESINATION_CODE,...,PICKUP_SCHEDULE_DATE_TIME,SHIPPER_DISTRICT,DURATION,SLA_DATE,SLA_GRADING,AGING,AGING_CATEGORY,PROPOSE_CANCEL,Total Pickup,COURIER_INSENTIVE
0,5265362112,Non COD,Tokopedia,MMJ TOOLS,"Komplek ruko harmoni mas E , jln jembatan Pe...",Kota Administrasi Jakarta Utara,DKI Jakarta,Penjaringan,Administrasi Jakarta Utara,CGU,...,2025-04-24 23:59:00,"Komplek ruko harmoni mas E , jln jembatan Pe...",0.0,2025-04-26,Meet SLA,NaN,NaN,NaN,183,20000
1,5265363308,Non COD,Tokopedia,Rumix,"Ruko Mangga Dua Square, Pademangan, Jakaa Utar...",Kota Administrasi Jakarta Utara,DKI Jakarta,Kemayoran,Administrasi Jakarta Pusat,CGP,...,2025-04-24 23:59:00,"Ruko Mangga Dua Square, Pademangan, Jakaa Utar...",0.0,2025-04-26,Meet SLA,NaN,NaN,NaN,82,0
2,5265382814,COD,Tokopedia,Albatha Herbal Nusantara,(Pos Sekuriti Pintu Masuk) Apt Wisma Gading Pe...,Kota Administrasi Jakarta Utara,DKI Jakarta,Kelapa Gading,Administrasi Jakarta Utara,CGU,...,2025-04-24 23:59:00,(Pos Sekuriti Pintu Masuk) Apt Wisma Gading Pe...,NaN,2025-04-26,Over SLA,52.0,>14,NaN,0,0
3,2448345410,Non COD,Kurir Rekomendasi,BenuaCell,JL DUTA HARAPAN INDAH V V A2/A22 /RE,Kota Administrasi Jakarta Utara,DKI Jakarta,Penjaringan,Administrasi Jakarta Utara,CGU,...,2025-04-24 23:59:00,JL DUTA HARAPAN INDAH V V A2/A22 /RE,0.0,2025-04-26,Meet SLA,NaN,NaN,NaN,215,30000
4,5265374504,Non COD,Tokopedia,BAHAGIA ELEKTRONIK,HARCO MANGGA DUA PLAZA RUKO P A,Kota Administrasi Jakarta Pusat,DKI Jakarta,Kemayoran,Administrasi Jakarta Pusat,CGP,...,2025-04-24 23:59:00,HARCO MANGGA DUA PLAZA RUKO P A,0.0,2025-04-26,Meet SLA,NaN,NaN,NaN,32,0


In [71]:

courier_insentif_new = df.groupby(['NAMA_SIGESIT','PICKUP_DATE_TIME','NAMA_GERAI']).agg({
    'COURIER_INSENTIVE':'sum',
    'Total Pickup':'mean'
}).reset_index()
courier_insentif_new['Total Pickup'] = courier_insentif_new['Total Pickup'].astype(int)
courier_insentif_new.to_parquet("courier_insentif_new.parquet")
courier_insentif_new

,NAMA_SIGESIT,PICKUP_DATE_TIME,NAMA_GERAI,COURIER_INSENTIVE,Total Pickup
0,Ronal Satria Perdana,2025-04-23 00:00:00,Jakbar Kembangan FM,0,1
1,Ronal Satria Perdana,2025-04-27 00:00:00,Jakbar Kembangan FM,0,1
2,A Romadoni,2025-04-21 00:00:00,Jakbar Angke FM,0,10
3,A Romadoni,2025-04-22 00:00:00,Jakbar Angke FM,0,20
4,A Romadoni,2025-04-23 00:00:00,Jakbar Angke FM,0,24
...,...,...,...,...,...
5196,muhammad adli naufal,2025-04-24 00:00:00,Jakut Bandengan Raya FM,30000,329
5197,muhammad adli naufal,2025-04-25 00:00:00,Jakut Bandengan Raya FM,0,23
5198,muhammad adli naufal,2025-04-26 00:00:00,Jakut Bandengan Raya FM,0,53
5199,muhammad adli naufal,2025-04-28 00:00:00,Jakut Bandengan Raya FM,0,37


In [ ]:
pickup_perfomance = df.groupby(['PICKUP_DATE_TIME','NAMA_GERAI','SLA_GRADING']).size().unstack(fill_value=0).reset_index()
pickup_perfomance['Total'] = pickup_perfomance['Meet SLA'] + pickup_perfomance['Over SLA']
pickup_perfomance['% SLA'] = pickup_perfomance['Meet SLA']/pickup_perfomance['Total']
pickup_perfomance.sort_values(by='Total', ascending=False)
pickup_perfomance1 = pickup_perfomance.drop(columns=['Meet SLA','Over SLA','Total'])

pickup_perfomance1.to_parquet("pickup_perfomance1.parquet")
